In [1]:
import sys
from pathlib import Path

SRC = (Path.cwd() / "src").resolve()
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
print("sys.path OK:", sys.path[0])


sys.path OK: C:\Users\maitha.alhammadi\projects\RL_2025\src


In [2]:
%pip install -U pip setuptools wheel
%pip install "spacy==3.7.6"
!python -m spacy download en_core_web_md
%pip install -q nest_asyncio

# make sure it's installed for THIS kernel
%pip install -q nest_asyncio

import asyncio
import nest_asyncio

# allow re-entering the running loop (the magic sauce)
nest_asyncio.apply()

def run_async(coro):
    """
    Safely run an async coroutine both:
    - in a normal Python script, and
    - inside a Jupyter/VS Code notebook (already-running loop)
    """
    try:
        # Works in plain scripts
        return asyncio.run(coro)
    except RuntimeError:
        # Notebook: loop is already running
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(coro)




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     ---------------------------------------- 0.3/42.8 MB ? eta -:--:--
     ---------------------------------------- 0.5/42.8 MB 1.9 MB/s eta 0:00:23
      --------------------------------------- 1.0/42.8 MB 1.8 MB/s eta 0:00:24
     - -------------------------------------- 1.3/42.8 MB 1.8 MB/s eta 0:00:24
     - -------------------------------------- 1.6/42.8 MB 1.7 MB/s eta 0:00:25
     - -------------------------------------- 2.1/42.8 MB 1.8 MB/s eta 0:00:24
     -- ------------------------------------- 2.4/42.8 MB 1.7 MB/s eta 0:00:24
     -- ------------------------------------- 2.6/42.8 MB 1.7 MB/s eta 0:00:24
     -- ------------------------------------- 3.1/42.8 MB 1.7 MB/s eta 0:00:23
     --- ------------------------------------ 3.4/42.8 MB 1.7 MB/s eta 0:00:24
     --- ----

In [3]:
import groq, pdfplumber, spacy, nest_asyncio
from sentence_transformers import SentenceTransformer

print("groq ok:", groq.__version__)
print("pdfplumber ok")
print("spacy ok:", spacy.__version__)
print("sbert ok:", SentenceTransformer.__name__)


groq ok: 0.20.0
pdfplumber ok
spacy ok: 3.7.6
sbert ok: SentenceTransformer


In [4]:
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv(dotenv_path=Path.cwd() / ".env")
print("Key:", os.getenv("GROQ_API_KEY"))

Key: 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


In [5]:
# --- make src/ importable ---
import sys, os
from pathlib import Path

SRC = (Path.cwd() / "src").resolve()
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print("sys.path OK:", sys.path[0])

# --- backend imports (your 4 RL files) ---
from backend.state import encode_state
from backend.rl_agent import RLAgent
from backend.reward import feedback_reward
from backend.retrieval_policies import action_to_filter

# --- RAG + LLM (your existing code) ---
from backend.rag_process import rag_process
from backend.ghg_assistant import GHGAssistant

print("Backend modules loaded.")


sys.path OK: C:\Users\maitha.alhammadi\projects\RL_2025\src


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maitha.alhammadi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


GROQ key prefix: gsk_WQGM ...OK
Backend modules loaded.


In [6]:
# If your assistant requires GROQ_API_KEY in env:
try:
    from dotenv import load_dotenv
    # load .env at repo root if present
    load_dotenv(dotenv_path=Path.cwd() / ".env")
except Exception:
    pass

assert os.getenv("GROQ_API_KEY"), "GROQ_API_KEY not found. export it or put it in .env"
print("Key prefix:", os.getenv("GROQ_API_KEY")[:8], "...OK")


Key prefix: gsk_WQGM ...OK


In [7]:
import asyncio
import nest_asyncio

def run_async(coro):
    """
    Run an async coroutine safely from a notebook OR a normal script.
    - In a notebook (loop already running): reuse it via nest_asyncio.
    - In a script: create a fresh loop with asyncio.run.
    """
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None

    if loop and loop.is_running():
        nest_asyncio.apply(loop)            # patch the running loop once
        return loop.run_until_complete(coro)
    else:
        return asyncio.run(coro)


In [8]:
# --- make src/ importable ---
import sys
from pathlib import Path

SRC = (Path.cwd() / "src").resolve()
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

# --- imports from backend ---
from backend.rl_agent import RLAgent
from backend.rag_process import rag_process
from backend.ghg_assistant import GHGAssistant


In [9]:

# RL agent persists its Q-table under src/data/q_table.json
agent = RLAgent()

# Your RAG layer + assistant (as in your repo)
rag = rag_process()
assistant = GHGAssistant()

print("Agent, RAG and Assistant are ready.")


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Agent, RAG and Assistant are ready.


In [10]:
from typing import List, Dict, Any, Tuple

def retrieve_chunks(question: str, action: str, company_name: str | None) -> Tuple[List[str], List[Dict[str, Any]]]:
    """Converts the chosen action into a metadata filter and queries your RAG."""
    meta = action_to_filter(action, company_name)
    try:
        # If you already extended `query_documents` to accept metadata_filter
        chunks, metas = rag.query_documents(question=question, n_results=4, metadata_filter=meta)
    except TypeError:
        # Backward-compatible path if your method doesn't take metadata_filter yet
        chunks, metas = rag.query_documents(question=question, n_results=4)
    return chunks, metas

def format_context(chunks: List[str], metas: List[Dict[str, Any]]) -> str:
    """Optional: pretty join chunks + sources."""
    formatted = []
    for ch, md in zip(chunks, metas):
        src = md.get("source", "Unknown")
        pg  = md.get("page") or md.get("chunk_number")
        tag = f"{src}" + (f" (p.{pg})" if pg else "")
        formatted.append(f"Source: {tag}\n{ch}")
    return "\n\n---\n\n".join(formatted)


In [11]:
# We'll keep the last (state, action) so you can send 👍/👎 afterwards
last_step = {
    "state": None,
    "action": None,
    "company": None,
    "response": None
}


In [12]:
def ask(prompt: str, company_info: dict | None = None) -> str:
    """
    1) Build RL state from prompt + company_info
    2) Choose action via RL
    3) Retrieve chunks respecting the action (filter)
    4) Ask the assistant
    5) Cache state/action so you can call thumb_up()/thumb_down() afterwards
    """
    # 1) state
    s = encode_state(prompt, company_info or {})

    # 2) RL action
    a = agent.select(s)

    # 3) retrieval
    company_name = (company_info or {}).get("name")
    chunks, metas = retrieve_chunks(prompt, a, company_name)
    context = format_context(chunks, metas)

    # 4) assistant
    # Your GHGAssistant uses: await generate_response(user_prompt, context)
    # This wrapper calls it synchronously.
    import asyncio
    async def _call():
        return await assistant.generate_response(user_prompt=prompt, context=context)
    try:
        answer = asyncio.run(_call())
    except RuntimeError:
        # If there's already an event loop (sometimes happens in notebooks)
        answer = asyncio.get_event_loop().run_until_complete(_call())

    # 5) remember for feedback
    last_step.update(state=s, action=a, company=company_info, response=answer)

    print(f"[Action] {a}")
    return answer


In [13]:
from datetime import datetime
import csv

LOG_PATH = (SRC / "data" / "rl_logs.csv")
LOG_PATH.parent.mkdir(parents=True, exist_ok=True)

def _log_feedback(state, action, tag):
    LOG_PATH.parent.mkdir(parents=True, exist_ok=True)
    new_file = not LOG_PATH.exists()
    with LOG_PATH.open("a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        if new_file:
            w.writerow(["ts", "action", "tag"])
        w.writerow([datetime.utcnow().isoformat(), action, tag])

def thumb_up():
    assert last_step["state"] is not None, "Call ask(...) first."
    r = feedback_reward("up")
    agent.update(last_step["state"], last_step["action"], r)
    _log_feedback(last_step["state"], last_step["action"], "up")
    print("👍 thanks — learning updated.")

def thumb_down():
    assert last_step["state"] is not None, "Call ask(...) first."
    r = feedback_reward("down")
    agent.update(last_step["state"], last_step["action"], r)
    _log_feedback(last_step["state"], last_step["action"], "down")
    print("👎 got it — learning updated.")


In [14]:
# --- make src/ importable ---
import sys
from pathlib import Path

SRC = (Path.cwd() / "src").resolve()
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

# --- backend imports ---
from backend.state import encode_state
from backend.rl_agent import RLAgent
from backend.reward import feedback_reward
from backend.retrieval_policies import action_to_filter   # <- this fixes the error
from backend.rag_process import rag_process
from backend.ghg_assistant import GHGAssistant

# --- init singletons ---
agent = RLAgent()
rag = rag_process()
assistant = GHGAssistant()

print("Backend modules loaded.")


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Backend modules loaded.


In [15]:
from backend.rl_agent import RLAgent
from backend.pipeline import answer_with_rl
from backend.reward import feedback_reward  # your thumbs → ±1 mapper

agent = RLAgent(verbose=True)  # keep verbose to see [Exploit]/[Explore] + [Update]
company = {"name": "Acme", "sector": "energy", "size": "large"}

ans, action, chunks, metas, s = answer_with_rl(
    "What penalties apply under NGER non-compliance?",
    company,
    agent
)
print("ACTION:", action)
print("ANSWER:", ans[:800])

# Provide feedback (simulate thumbs)
r = feedback_reward("up")   # "up" or "down"
agent.update(s, action, r, s)  # next_state = s is OK if you don't change state

[Exploit] Best action: legal_only with Q=1.147


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


ACTION: legal_only
ANSWER: Under the National Greenhouse and Energy Reporting (NGER) scheme, non-compliance can result in penalties. The Clean Energy Regulator (CER) is responsible for enforcing compliance. 

For corporations, the maximum penalty for non-compliance is 12,600 penalty units for each offense, which currently translates to AUD 2,664,000 (1 penalty unit = AUD 222). 

For individuals, the maximum penalty is 240 penalty units or AUD 53,280, and/or 6 months imprisonment for each offense.

Additionally, failure to comply with NGER requirements can lead to infringement notices, with a penalty of 20 penalty units for corporations (AUD 4,440) and 4 penalty units for individuals (AUD 888).

It's crucial for Australian companies to maintain accurate records, submit reports on time, and ensure data quality to avoi
[Update] State={"len": "short", "month": "2025-10", "sector": "energy", "size": "large", "topic": "legal"}, Action=legal_only, OldQ=1.147, Reward=1.0, Target=2.032, NewQ=1.

In [16]:
from backend.rl_agent import RLAgent
from backend.state import encode_state

agent = RLAgent(verbose=True)

company = {"name":"Acme","sector":"energy","size":"large"}
s = encode_state("How do we cut Scope 3 next year?", company)

a = agent.select(s)
r = +1.0
agent.update(s, a, r, s)

print("Q row:", agent.q_for(s))
print("Best action:", agent.best_action(s))
answer = ask("Can you give me steps of calculating pneumatic punp emissions in Australia?", company)
print("\nAssistant:\n", answer)

[Exploit] Best action: company_only with Q=0.300
[Update] State={"len": "short", "month": "2025-10", "sector": "energy", "size": "large", "topic": "ghg"}, Action=company_only, OldQ=0.300, Reward=1.0, Target=1.270, NewQ=0.591
Q row: {'broad': -0.09000000000000002, 'legal_only': 0.19470000000000004, 'financial_only': 0.0, 'company_only': 0.591}
Best action: company_only
[Exploit] Best action: company_only with Q=0.591
[Action] company_only

Assistant:
 To calculate emissions from pneumatic pumps in Australia, follow these steps:

1. **Determine the type of fuel used**: Identify the fuel source powering the pneumatic pumps, such as electricity, natural gas, or diesel. This will help you calculate the emissions under the correct scope (Scope 1 for direct emissions or Scope 2 for indirect emissions from electricity use).

2. **Measure energy consumption**: Record the energy consumption of the pneumatic pumps in units of energy relevant to the fuel type (e.g., kilowatt-hours (kWh) for electr

In [18]:
thumb_down()

[Update] State={"len": "short", "month": "2025-10", "sector": "energy", "size": "large", "topic": "ghg"}, Action=company_only, OldQ=0.114, Reward=-1.0, Target=-1.000, NewQ=-0.220
👎 got it — learning updated.


In [19]:
answer = ask("What are the financial risks of carbon pricing for us?", company)
print("\nAssistant:\n", answer)


[Exploit] Best action: legal_only with Q=0.195
[Action] legal_only

Assistant:
 As an Australian company, you are likely subject to the Australian government's climate change policies, including the Safeguard Mechanism and potential future carbon pricing mechanisms. The financial risks of carbon pricing for your company can be significant and should be carefully assessed.

**Direct Costs:**

1. **Safeguard Mechanism**: If your company has facilities with emissions above 100,000 tonnes CO2-e per year, you may be required to surrender Australian Carbon Credit Units (ACCUs) or pay a fixed charge for excess emissions. This can result in direct costs to your business.
2. **Fuel Tax and Other Levies**: Changes to fuel tax and other levies can increase the cost of energy and transportation for your company.

**Indirect Costs:**

1. **Increased Energy Costs**: Carbon pricing can lead to higher energy costs, as electricity and fuel generators pass on the cost of emissions to consumers.
2. **Sup

In [20]:
thumb_up()    # or: thumb_down()


[Update] State={"len": "short", "month": "2025-10", "sector": "energy", "size": "large", "topic": "ghg"}, Action=legal_only, OldQ=0.195, Reward=1.0, Target=1.000, NewQ=0.436
👍 thanks — learning updated.


In [36]:
answer = ask("How can companies ensure the accuracy and consistency of their GHG emissions data, and what are the consequences of inaccurate reporting?", company)
print("\nAssistant:\n", answer)


[Exploit] Best action: broad with Q=0.000
[Action] broad

Assistant:
 To ensure the accuracy and consistency of GHG emissions data, Australian companies can follow these steps:

1. **Establish a robust data collection system**: Implement a reliable and transparent data collection process, using primary data sources, such as meter readings, fuel consumption records, and production data.
2. **Use accepted methodologies and factors**: Apply the National Greenhouse and Energy Reporting (NGER) Scheme's methodologies and factors, as well as the Australian Government's National Inventory Report, to calculate emissions.
3. **Conduct regular data quality checks**: Verify data for completeness, accuracy, and consistency, and address any discrepancies or gaps.
4. **Engage independent review and assurance**: Consider hiring external experts to review and assure the accuracy of emissions data, as required by the Corporations Act 2001 and the Australian Securities and Investments Commission (ASIC) g

In [37]:
thumb_up()


[Update] State={"len": "medium", "month": "2025-10", "sector": "energy", "size": "large", "topic": "ghg"}, Action=broad, OldQ=0.000, Reward=1.0, Target=1.000, NewQ=0.300
👍 thanks — learning updated.


In [23]:
answer = ask("How to increase the company profit", company)
print("\nAssistant:\n", answer)


[Exploit] Best action: company_only with Q=0.300
[Action] company_only

Assistant:
 This digital consultant specializes in Australian GHG emission regulations. Please rephrase your question to focus on topics such as compliance, emission calculations, or scope definitions related to GHG emissions.


In [24]:
thumb_up()


[Update] State={"len": "short", "month": "2025-10", "sector": "energy", "size": "large", "topic": "fin"}, Action=company_only, OldQ=0.300, Reward=1.0, Target=1.000, NewQ=0.510
👍 thanks — learning updated.


In [25]:
answer = ask("What financial risks do carbon taxes pose", company)
print("\nAssistant:\n", answer)


[Exploit] Best action: legal_only with Q=0.436
[Action] legal_only

Assistant:
 Under Australia's climate disclosure regulations, effective from 2025, companies must consider the financial risks associated with carbon taxes. The Australian government has implemented various policies to reduce greenhouse gas emissions, including the Safeguard Mechanism, which imposes a carbon tax on large emitters.

The financial risks posed by carbon taxes to Australian companies include:

1. **Increased operational costs**: Carbon taxes can increase the cost of production, transportation, and other business operations, potentially reducing profit margins.
2. **Compliance costs**: Companies may need to invest in new technologies, processes, or systems to reduce their emissions and comply with carbon tax regulations, incurring additional costs.
3. **Revenue risks**: Carbon taxes can lead to higher prices for goods and services, potentially reducing demand and impacting revenue.
4. **Competitiveness risk

In [27]:
thumb_up()


[Update] State={"len": "short", "month": "2025-10", "sector": "energy", "size": "large", "topic": "ghg"}, Action=legal_only, OldQ=0.605, Reward=1.0, Target=1.000, NewQ=0.724
👍 thanks — learning updated.


In [28]:
ask("Suggest the most cost-effective emission reduction policy", company)
print("\nAssistant:\n", answer)

[Explore] Random action: financial_only
[Action] financial_only

Assistant:
 Under Australia's climate disclosure regulations, effective from 2025, companies must consider the financial risks associated with carbon taxes. The Australian government has implemented various policies to reduce greenhouse gas emissions, including the Safeguard Mechanism, which imposes a carbon tax on large emitters.

The financial risks posed by carbon taxes to Australian companies include:

1. **Increased operational costs**: Carbon taxes can increase the cost of production, transportation, and other business operations, potentially reducing profit margins.
2. **Compliance costs**: Companies may need to invest in new technologies, processes, or systems to reduce their emissions and comply with carbon tax regulations, incurring additional costs.
3. **Revenue risks**: Carbon taxes can lead to higher prices for goods and services, potentially reducing demand and impacting revenue.
4. **Competitiveness risks**

In [29]:
thumb_down()

[Update] State={"len": "short", "month": "2025-10", "sector": "energy", "size": "large", "topic": "legal"}, Action=financial_only, OldQ=0.000, Reward=-1.0, Target=-1.000, NewQ=-0.300
👎 got it — learning updated.


In [30]:
ask("what are GHG regulations in the UK", company)
print("\nAssistant:\n", answer)

[Exploit] Best action: legal_only with Q=1.413
[Action] legal_only

Assistant:
 Under Australia's climate disclosure regulations, effective from 2025, companies must consider the financial risks associated with carbon taxes. The Australian government has implemented various policies to reduce greenhouse gas emissions, including the Safeguard Mechanism, which imposes a carbon tax on large emitters.

The financial risks posed by carbon taxes to Australian companies include:

1. **Increased operational costs**: Carbon taxes can increase the cost of production, transportation, and other business operations, potentially reducing profit margins.
2. **Compliance costs**: Companies may need to invest in new technologies, processes, or systems to reduce their emissions and comply with carbon tax regulations, incurring additional costs.
3. **Revenue risks**: Carbon taxes can lead to higher prices for goods and services, potentially reducing demand and impacting revenue.
4. **Competitiveness risk

In [31]:
thumb_down()

[Update] State={"len": "short", "month": "2025-10", "sector": "energy", "size": "large", "topic": "legal"}, Action=legal_only, OldQ=1.413, Reward=-1.0, Target=-1.000, NewQ=0.689
👎 got it — learning updated.


In [34]:
ask("“How can our company transition to net-zero by 2050?", company)
print("\nAssistant:\n", answer)

[Exploit] Best action: legal_only with Q=0.000
[Action] legal_only

Assistant:
 Under Australia's climate disclosure regulations, effective from 2025, companies must consider the financial risks associated with carbon taxes. The Australian government has implemented various policies to reduce greenhouse gas emissions, including the Safeguard Mechanism, which imposes a carbon tax on large emitters.

The financial risks posed by carbon taxes to Australian companies include:

1. **Increased operational costs**: Carbon taxes can increase the cost of production, transportation, and other business operations, potentially reducing profit margins.
2. **Compliance costs**: Companies may need to invest in new technologies, processes, or systems to reduce their emissions and comply with carbon tax regulations, incurring additional costs.
3. **Revenue risks**: Carbon taxes can lead to higher prices for goods and services, potentially reducing demand and impacting revenue.
4. **Competitiveness risk

In [35]:
thumb_down()

[Update] State={"len": "short", "month": "2025-10", "sector": "energy", "size": "large", "topic": "other"}, Action=legal_only, OldQ=0.000, Reward=-1.0, Target=-1.000, NewQ=-0.300
👎 got it — learning updated.


In [38]:
ask("What role do science-based targets play in a company's climate strategy, and how can they be effectively set and implemented?", company)
print("\nAssistant:\n", answer)

[Explore] Random action: legal_only
[Action] legal_only

Assistant:
 To ensure the accuracy and consistency of GHG emissions data, Australian companies can follow these steps:

1. **Establish a robust data collection system**: Implement a reliable and transparent data collection process, using primary data sources, such as meter readings, fuel consumption records, and production data.
2. **Use accepted methodologies and factors**: Apply the National Greenhouse and Energy Reporting (NGER) Scheme's methodologies and factors, as well as the Australian Government's National Inventory Report, to calculate emissions.
3. **Conduct regular data quality checks**: Verify data for completeness, accuracy, and consistency, and address any discrepancies or gaps.
4. **Engage independent review and assurance**: Consider hiring external experts to review and assure the accuracy of emissions data, as required by the Corporations Act 2001 and the Australian Securities and Investments Commission (ASIC) gu

In [39]:
thumb_down()

[Update] State={"len": "medium", "month": "2025-10", "sector": "energy", "size": "large", "topic": "other"}, Action=legal_only, OldQ=0.000, Reward=-1.0, Target=-1.000, NewQ=-0.300
👎 got it — learning updated.
